<img src="img/dsci511_header.png" width="600">

# Lecture 4: Plotting, Merging and Grouping

## Lecture Learning Objectives

- Make basic plots in Pandas by accessing the `.plot` attribute or importing functions from `pandas.plotting`.
- Combine dataframes using `df.merge()` and `pd.concat()` and know when to use these different methods.
- Apply functions to a dataframe or Series `DataFrame.apply()` and `DataFrame.applymap()` and `Series.map()`
- Perform grouping and aggregating operations using `df.groupby()` and `df.agg()`.
- Perform aggregating methods on grouped or ungrouped objects such as finding the minimum, maximum and sum of values in a dataframe using `df.agg()`.

In [1]:
import numpy as np
import pandas as pd

## Visualizing your data

You will study data visualization in depth in DSCI 531. However, it is good to know that pandas provides some basic plotting functionality-- you can use this to take a quick look at your data.

This feature of pandas is accessed via the `plot()` method. It uses the `matplotlib` library, which you can install with `conda install matplotlib` as you have done with other packages.

Once we have `matplotlib` installed, let's load in the dataset `'data/cycling_data.csv'` which includes some dates, times and comments about bike rides.

In [2]:
bike_df = pd.read_csv('data/cycling_data.csv', index_col=0, parse_dates=True).dropna()
bike_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/cycling_data.csv'

Let's go ahead and make a plot of the distances ridden.

In [3]:
bike_df['Distance'].plot()

NameError: name 'bike_df' is not defined

Notice the line graph joined adjacent points together, which may suggest there was a non-zero riding distance on (say) Sep 13 or Sep 21. But there are no biking entries for these days!

Plotting with pandas can be a good way to take a quick look at your data. It can also help you to spot potential problems. For example, let's make a graph of *cumulative* distance covered over time. 

In [4]:
bike_df['Distance'].cumsum().plot()

NameError: name 'bike_df' is not defined

That kink in the graph is funny... since we're plotting cumulative distance, the line should always be moving up and to the right! 

In [5]:
bike_df.loc['2019-09-18']

NameError: name 'bike_df' is not defined

Looks like some entries were out of chronological order. We should sort the index before plotting.

In [6]:
bike_df = bike_df.sort_index()
bike_df['Distance'].cumsum().plot()

NameError: name 'bike_df' is not defined

That's better! We can also change plot parameters like colour and line thickness.

In [7]:
bike_df['Distance'].cumsum().plot.line(fontsize=8, linewidth=3, color='r', ylabel='km');

NameError: name 'bike_df' is not defined

There are many other kinds of plots you can make:

|Method|Plot Type|
|---|---|
|`bar` or `barh` | bar plots|
|`hist` | histogram|
|`box` | boxplot|
|`kde` or `density` | density plots|
|`area` | area plots|
|`scatter` | scatter plots|
|`hexbin` | hexagonal bin plots|
|`pie` | pie plots|

Plus, there are more advanced plotting functions (scatter matrix, lag plots and others) that can be accessed via the `pandas.plotting` module.

## Combining multiple data frames

Sometimes we have meaningful data spread across different data frames (or read in from different files) that we wish to combine. There are many ways to combine data in pandas, and we will look at some of them today.

To begin let's load in some data frames with weather information for different dates. You might notice that these frames share information across certain rows or columns, i.e. the data sets are related.

In [8]:
w1 = pd.read_csv('data/weather1.csv', index_col=0)
w2 = pd.read_csv('data/weather2.csv', index_col=0)
w3 = pd.read_csv('data/weather3.csv', index_col=0)
w4 = pd.read_csv('data/weather4.csv', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'data/weather1.csv'

In [9]:
w1 # Let's look at these Data Frames

NameError: name 'w1' is not defined

In [10]:
w2

NameError: name 'w2' is not defined

In [11]:
w3

NameError: name 'w3' is not defined

In [12]:
w4

NameError: name 'w4' is not defined

### Combining with `pd.concat()`

The simplest way to combine Data Frames in pandas is to use `pd.concat()`, which concatenates them along a shared axis. The default is to concatenate rows:

In [13]:
pd.concat([w1,w2])

NameError: name 'w1' is not defined

The above worked well since both Data Frames `w1` and `w2` have the same set of columns. Let's see what happens when we have a few different columns:

In [14]:
pd.concat([w1,w4])

NameError: name 'w1' is not defined

We get a new frame containing columns from both `w1` and `w4`. The common columns are concatenated, and missing data shows up as `NaN`. Notice that pandas did not know that the indices were 'common', i.e. indices 2 and 3 are repeated in the output.

Next, let's concatenate two Data Frames with shared rows but different columns:

In [15]:
pd.concat([w1,w3], axis = 1)

NameError: name 'w1' is not defined

And again when only some rows are shared:

In [16]:
pd.concat([w1,w4], axis = 1)

NameError: name 'w1' is not defined

This time pandas knew to use a shared index for rows, but the column indices were treated independently. As a result, we have "duplicate" columns in the output.

We can concatenate as many Data Frames as we want, but each call to `pd.concat` creates a new copy of the data frame. So instead of copying 'one-at-a-time', the best practice is to assemble all required Data Frames into a single list, and only call `pd.concat` once.

In [17]:
pd.concat([w1,w2,w3,w4])

NameError: name 'w1' is not defined

We were certainly able to "stick" our data frames together here, but we may not have gained the information we wanted. For example, the data frame above contains entries for both `Mean Temp` and `Total Rain` for November 12, 2012, but these are split across different rows.

If we know beforehand that shared dates should correspond to the same example in our data, it would be good to use this fact when combining the data frames. The next section discuss how to implement this in pandas.

### Combining with `pd.Merge`

Another pandas function that is useful for joining Data Frames together is `pd.merge`. This function implements SQL-style joins in pandas. You will learn more about SQL when you study databases in DSCI 513. `pd.merge()`.

When using `pd.concat()`, you have to do all the work beforehand to make sure the rows and/or columns line up just right for the concatenation to give the correct, meaningful result. This can be tricky when working with large Data Frames. With `pd.merge`, we can pass a little more meaningful information to pandas which it uses combine our Data Frames to get the result we intended. Essentially, the matching is done based on columns shared between the Data Frames.

In [18]:
pd.merge(w1, w4, how='outer', on = ['Date/Time', 'Month', 'Year'])

NameError: name 'w1' is not defined

This time, pandas knew that rows with matching Date/Time, Month and Year should count as the same observation for both frames. But notice that the index was ignored! The resulting Data Frame has an entirely new index, since we were matching on columns. Matching on the index is also possible.

In [19]:
pd.merge(w1, w4, how='outer', left_index = True, right_index = True)

NameError: name 'w1' is not defined

The `how='outer'` parameter tells pandas to produce a Data Frame containing the full list of keys from both Data Frames. We could restrict this to keys in only one Data Frame:

In [20]:
pd.merge(left = w1, right = w4, how='right', on = ['Date/Time', 'Month', 'Year'])

NameError: name 'w1' is not defined

Notice again that the index was not retained when matching on columns.

Let's create a Data Frame of 'seasons' to categorize the different months of the year.

In [21]:
seasons = pd.DataFrame({
                    'month':[1,2,3,4,5,6,7,8,9,10,11,12], 
                    'season': ['Winter', 'Winter', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Fall', 'Fall', 'Fall', 'Winter']
                       }
                      )

seasons

,month,season
0,1,Winter
1,2,Winter
2,3,Spring
3,4,Spring
4,5,Spring
5,6,Summer
6,7,Summer
7,8,Summer
8,9,Fall
9,10,Fall


We made this Data Frame "by hand", using a dictionary object. Dictionaries are another object type in Python, similar to lists but indexed based on "keys" that do not need to be integers. The keys in this case serve as column names when passed to `pd.DataFrame()`.

Now if we have some dataset that includes months as a column, we can merge with `seasons`. For example, we can use the following data set that we saw in Lecture 1 (we are dropping some columns):

In [22]:
weather = pd.read_csv('data/YVR_weather_data.csv').dropna(axis=1)
weather.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/YVR_weather_data.csv'


Let's attempt a left join for `weather` with `seasons`. We will have to tell pandas that `Month` in the left frame matches `month` in the second (note that the strings are not identical).

In [23]:
pd.merge(left = weather, right = seasons, how = 'left', left_on = 'Month', right_on = 'month')

NameError: name 'weather' is not defined

With `pd.merge()`, we can combine data frames in a one-to-one, one-to-many or a many-to-many fashion, leading to a large range of possibilities. You will see many examples of joins in DSCI 513.

### Applying Custom Functions

There will be times when you want to apply a function that is not built-in to Pandas. For this, we also have methods:

* `df.apply()`, applies a function **column-wise or row-wise** across a dataframe (the function must be able to accept/return an array)

* `df.map()`, applies a function element-wise (so individually) across **every element** in the data frame (meant for functions that accept/return single values at a time)
    
* `series.apply()`, same as above for Pandas `Series`.

* `series.map()`, also for Series, but optionally accepts a dictionary as input.

It is important to understand how `.apply()` and `.map()` differ. Sometimes it appears as though these functions work the same. This is the case for functions which accept and return single values at a time, like the `round` function:

<img src="img/apply-round.png" width=600>

<img src="img/map-round.png" width=600>

However, when using `.apply()` and `.map()` with methods/functions that act on a series or array and return a single value, like `np.sum`, the objects they return are quite different:

<img src="img/apply-sum.png" width=600>

<img src="img/map-sum.png" width=600>

It is also important to know how `pandas.DataFrame.apply()` works with regards to what `axis=0` and `axis=1` means. This clarification is important because in many other pandas functions/methods,  `axis=0` means row-wise, and  `axis=1` means column-wise. But that is not the case here. From the [`pandas.DataFrame.apply()` docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html):

*"Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1)."*

So for `pandas.DataFrame.apply()`:
- `axis=0` means the **columns** (saying you want the index to be the DataFrame’s index)
- `axis=1` means the **rows** (saying you want the index to be the DataFrame’s columns)

**For `pandas.DataFrame.apply()` `axis=0` is the default, meaning it works column-wise unless you specify otherwise (as shown in the examples above).**

Here's some examples of using `.apply()` and `.map()`  with our weather data:

In [24]:
weather[['Month', 'Mean Temp (°C)']].apply(np.log) # we will see more from numpy in future lectures

NameError: name 'weather' is not defined

In [25]:
weather['Mean Temp (°C)'].apply(int) # apply is a method for both Data Frames and Series

NameError: name 'weather' is not defined

We provide a dictionary as input to `Series.map`.

In [26]:
weather[['Month', 'Mean Temp (°C)']].apply(np.sum)

NameError: name 'weather' is not defined

In [27]:
seasons['season'].map(
                        {'Winter':'Cold', 'Spring':'Not Cold', 'Summer': 'Warm'}
                     )

0         Cold
1         Cold
2     Not Cold
3     Not Cold
4     Not Cold
5         Warm
6         Warm
7         Warm
8          NaN
9          NaN
10         NaN
11        Cold
Name: season, dtype: object

Note that `'Fall'` did not appear as a key in the dictionary supplied, and the output shows `NaN` for the corresponding entries.

### Grouping

Often we are interested in examining specific groups in our data. In pandas, `df.groupby()` allows us to group our data based on one or more variables. It is analgous to the `group_by` function in R

In [28]:
imdb = pd.read_csv('data/imdb.csv')
imdb.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/imdb.csv'

Let's group this Data Frame on `Released_Year`:

In [29]:
imdb.groupby(by = 'Released_Year')

NameError: name 'imdb' is not defined

What is a `DataFrameGroupBy` object? It's really just a dictionary of index-mappings! The keys represent the different groups, and each key is mapped to a list of indices corresponding to that group. We can look at them directly if we really wanted to:

In [30]:
imdb.groupby(by = 'Released_Year').groups

NameError: name 'imdb' is not defined

We can access a specific group using the `get_group()` method.

In [31]:
imdb.groupby(by='Released_Year').get_group(1995)

NameError: name 'imdb' is not defined

The usual thing to do, however, is to apply aggregation functions to the `groupby` object. For example, we could take the mean of the numeric columns. Use `numeric_only=True` to tell pandas to ignore non-numeric columns.

In [32]:
imdb.groupby(by='Released_Year').mean(numeric_only=True)

NameError: name 'imdb' is not defined

Even better would be to select the numeric columns ahead of time, and then call `grouby()` and `mean()`. We can apply multiple functions using `aggregate()`.

In [33]:
(
    imdb
    .loc[:, ['Released_Year', 'IMDB_Rating', 'Meta_score', 'No_of_Votes']]
    .groupby(by='Released_Year')
    .aggregate(['mean', 'sum', 'count'])
)

NameError: name 'imdb' is not defined

We could also apply different functions to different columns.

In [34]:
imdb.groupby(by="Released_Year").agg(
    {
        "Meta_score": ["max", "min", "mean"],
        "Gross": ["sum"]
    }
)

NameError: name 'imdb' is not defined

Later on, we will learn how to write our own Python functions. Such 'custom' functions can also be passed to `aggregate()`.

Note that we can pass multiple columns to `groupby()`. The groups will be indexed based on all combinations of entries that appear in the listed columns

In [35]:
imdb.groupby(by=["Released_Year", "Certificate"]).agg(
    {
        "Meta_score": ["mean"],
        "Gross": ["sum"]
    }
)

NameError: name 'imdb' is not defined

Finally, you can `aggregate()` on a non-grouped data frame as well. This is essentially what `DataFrame.describe()` does under the hood.

In [36]:
(
    imdb
    .loc[:, ['IMDB_Rating', 'Meta_score', 'No_of_Votes']]
    .agg(['mean', 'min', 'count'])
)

NameError: name 'imdb' is not defined